In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Context and Objective**

The Brazilian championship since 2006 is a championship of 20 teams that play against each other in two matches; one at each team's home.
Each victory counts 3 points, each draw 1 point and a defeat generates 0 points.
The team with the most points is crowned Champion, while the bottom 4 are relegated to Série B (where the best 4 will come for the following year).

In this notebook we will analyze the possibilities of Corinthians (the team I support) not being relegated after scoring just 22 points after 24 rounds, their worst campaign since the championship had this format.

# **Methodology**

First we made some point projections for Corinthians and the teams fighting against relegation (1,2). These projections were based on:

- performance of this year;
- maximum, average and minimum performance considering an entire championship;
- maximum, average and minimum performance considering only the last remaining rounds of any championship;
Thus creating 7 scenarios for each team.

Then we compared Corinthians' projections with the projections of these teams.

We also compared Corinthians' projections with the history of the teams that were in 17th place (first of the relegated teams).

(1) We define teams that are fighting against relegation as teams that, after the same number of games, are at most 6 points away from the relegation zone.

(2) These teams are, in order: 'Atletico-GO', 'Cuiabá', 'Vitoria', 'Corinthians', 'Fluminense', 'Athletico PR', 'Juventude', 'Bragantino','Criciúma' and 'Grêmio'.

(3) The 29th round was chosen as the cutoff because it was between this round and the 30th that this analysis began to be carried out.


In [3]:
path = '/content/drive/MyDrive/Data Science Portfolio/Brasileirao/'
rounds = 'rounds_full_table'
years_to_analyze = list(range(2003, 2024))
df_full_table = pd.read_csv(path + rounds + '.csv')

In [5]:
teams_fighting_relegation = {
    'Vasco': [37,29],
    'Criciuma': [36, 30],
    'Gremio': [35, 30],
    'Bragantino':[34, 30],
    'Juventude': [34, 30],
    'Fluminense': [33, 29],
    'Vitoria': [32, 30],
    'Corinthians': [32, 30],
    'Athletico-PR': [31, 28],
    'Cuiaba': [27, 29],
    'Atletico-GO': [22, 30]
}

In [6]:
df_full_table_after_2005 = df_full_table[df_full_table['year'] > 2005]

In [7]:
df_fighting_relegation_campaigns_after_actual_round = pd.DataFrame(columns=['year','team','points_actual_round','points_38','points_difference'])
for team in teams_fighting_relegation:
  actual_round = teams_fighting_relegation[team][1]
  df_team = df_full_table_after_2005[['year', 'round', 'team', 'points']]
  df_team = df_team[df_team['team'] == team]
  df_team_actual_round = df_team[df_team['round'] == actual_round]
  df_team_round_38 = df_team[df_team['round'] == 38]
  df_team_rounds = pd.merge(df_team_actual_round, df_team_round_38, on=['year', 'team'], suffixes=('_actual_round', '_38'))
  df_team_rounds['points_difference'] = df_team_rounds['points_38'] - df_team_rounds['points_actual_round']
  df_team_rounds = df_team_rounds[['year', 'team', 'points_actual_round', 'points_38', 'points_difference']]
  df_fighting_relegation_campaigns_after_actual_round = pd.concat([df_fighting_relegation_campaigns_after_actual_round, df_team_rounds])

In [8]:
df_fighting_relegation_campaigns_after_actual_round

,year,team,points_actual_round,points_38,points_difference
0,2006,Vasco,44,59,15
1,2007,Vasco,40,54,14
2,2008,Vasco,27,40,13
3,2010,Vasco,41,49,8
4,2011,Vasco,51,69,18
...,...,...,...,...,...
2,2012,Atletico-GO,23,30,7
3,2017,Atletico-GO,26,36,10
4,2020,Atletico-GO,36,50,14
5,2021,Atletico-GO,37,53,16


In [9]:
# prompt: save df_fighting_relegation_campaigns_after_actual_round in path in excel

df_fighting_relegation_campaigns_after_actual_round.to_excel(path + 'df_fighting_relegation_campaigns_after_actual_round.xlsx', index=False)


In [10]:
df_fighting_relegation_campaigns_after_actual_round[df_fighting_relegation_campaigns_after_actual_round['team']=='Corinthians']

,year,team,points_actual_round,points_38,points_difference
0,2006,Corinthians,35,53,18
1,2007,Corinthians,37,44,7
2,2009,Corinthians,42,52,10
3,2010,Corinthians,50,68,18
4,2011,Corinthians,54,71,17
5,2012,Corinthians,43,57,14
6,2013,Corinthians,40,50,10
7,2014,Corinthians,52,69,17
8,2015,Corinthians,64,81,17
9,2016,Corinthians,45,55,10


In [11]:
df_agg_fighting_relegation_campaigns_after_actual_round = df_fighting_relegation_campaigns_after_actual_round.groupby('team').agg({
    'points_38': ['min', 'max', 'mean'],
    'points_difference': ['min', 'max', 'mean']
})
df_agg_fighting_relegation_campaigns_after_actual_round.columns = ['_'.join(col) for col in df_agg_fighting_relegation_campaigns_after_actual_round.columns]

df_agg_fighting_relegation_campaigns_after_actual_round['round'] = [teams_fighting_relegation[team][1] for team in df_agg_fighting_relegation_campaigns_after_actual_round.index]
df_agg_fighting_relegation_campaigns_after_actual_round['actual_points'] = [teams_fighting_relegation[team][0] for team in df_agg_fighting_relegation_campaigns_after_actual_round.index]

df_agg_fighting_relegation_campaigns_after_actual_round['projection_this_year'] = (df_agg_fighting_relegation_campaigns_after_actual_round['actual_points']/df_agg_fighting_relegation_campaigns_after_actual_round['round']) * 38
df_agg_fighting_relegation_campaigns_after_actual_round['projection_38_min'] = (df_agg_fighting_relegation_campaigns_after_actual_round['points_38_min']/38) * (38 - df_agg_fighting_relegation_campaigns_after_actual_round['round']) + df_agg_fighting_relegation_campaigns_after_actual_round['actual_points']
df_agg_fighting_relegation_campaigns_after_actual_round['projection_38_mean'] = (df_agg_fighting_relegation_campaigns_after_actual_round['points_38_mean']/38) * (38 - df_agg_fighting_relegation_campaigns_after_actual_round['round']) + df_agg_fighting_relegation_campaigns_after_actual_round['actual_points']
df_agg_fighting_relegation_campaigns_after_actual_round['projection_38_max'] = (df_agg_fighting_relegation_campaigns_after_actual_round['points_38_max']/38) * (38 - df_agg_fighting_relegation_campaigns_after_actual_round['round']) + df_agg_fighting_relegation_campaigns_after_actual_round['actual_points']
df_agg_fighting_relegation_campaigns_after_actual_round['projection_points_difference_min'] = df_agg_fighting_relegation_campaigns_after_actual_round['points_difference_min'] + df_agg_fighting_relegation_campaigns_after_actual_round['actual_points']
df_agg_fighting_relegation_campaigns_after_actual_round['projection_points_difference_mean'] = df_agg_fighting_relegation_campaigns_after_actual_round['points_difference_mean'] + df_agg_fighting_relegation_campaigns_after_actual_round['actual_points']
df_agg_fighting_relegation_campaigns_after_actual_round['projection_points_difference_max'] = df_agg_fighting_relegation_campaigns_after_actual_round['points_difference_max'] + df_agg_fighting_relegation_campaigns_after_actual_round['actual_points']
df_agg_fighting_relegation_campaigns_after_actual_round.reset_index(inplace=True)
df_agg_fighting_relegation_campaigns_after_actual_round = df_agg_fighting_relegation_campaigns_after_actual_round[['team','actual_points','projection_this_year','projection_38_min','projection_38_mean','projection_38_max','projection_points_difference_min','projection_points_difference_mean','projection_points_difference_max']]

for column in df_agg_fighting_relegation_campaigns_after_actual_round.columns:
  if column != 'team':
    df_agg_fighting_relegation_campaigns_after_actual_round[column] = df_agg_fighting_relegation_campaigns_after_actual_round[column].astype(int)

<ipython-input-11-cdc0705fdeee>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_agg_fighting_relegation_campaigns_after_actual_round[column] = df_agg_fighting_relegation_campaigns_after_actual_round[column].astype(int)
<ipython-input-11-cdc0705fdeee>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_agg_fighting_relegation_campaigns_after_actual_round[column] = df_agg_fighting_relegation_campaigns_after_actual_round[column].astype(int)
<ipython-input-11-cdc0705fdeee>:22: SettingWithCopyWarning: 
A v

In [12]:
df_agg_fighting_relegation_campaigns_after_actual_round

,team,actual_points,projection_this_year,projection_38_min,projection_38_mean,projection_38_max,projection_points_difference_min,projection_points_difference_mean,projection_points_difference_max
0,Athletico-PR,31,42,41,45,47,42,46,53
1,Atletico-GO,22,27,28,30,33,28,32,38
2,Bragantino,34,43,43,45,47,40,42,49
3,Corinthians,32,40,41,44,49,38,44,50
4,Criciuma,36,45,42,44,45,38,44,50
5,Cuiaba,27,35,36,37,39,36,38,41
6,Fluminense,33,43,43,46,51,37,46,54
7,Gremio,35,44,44,47,50,41,47,53
8,Juventude,34,43,38,42,43,36,42,47
9,Vasco,37,48,46,48,53,45,49,55


In [13]:
df_round_38 = df_full_table_after_2005[df_full_table_after_2005['round'] == 38]
df_position_17 = df_round_38[df_round_38['position'] == 17].reset_index(drop=True)
df_position_17 = df_position_17[['year', 'team', 'points']]
df_position_17['points_per_match'] = (df_position_17['points']/38)
df_position_17['average'] = df_position_17['points'].mean()

In [14]:
df_position_17

,year,team,points,points_per_match,average
0,2006,Ponte Preta,39,1.026316,41.666667
1,2007,Corinthians,44,1.157895,41.666667
2,2008,Figueirense,44,1.157895,41.666667
3,2009,Coritiba,45,1.184211,41.666667
4,2010,Vitoria,42,1.105263,41.666667
5,2011,Athletico-PR,41,1.078947,41.666667
6,2012,Sport,41,1.078947,41.666667
7,2013,Fluminense,46,1.210526,41.666667
8,2014,Vitoria,38,1.000000,41.666667
9,2015,Avai,42,1.105263,41.666667
